# Cтохастический градиентный спуск в задачах регрессии

Отличается от простой линейно модели тем, что применяет для подстройки весов [Алгоритм Градиентного Спуска](https://ru.wikipedia.org/wiki/%D0%93%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%BF%D1%83%D1%81%D0%BA).


![Градиентный спуск](https://upload.wikimedia.org/wikipedia/commons/7/79/Gradient_descent.png "Градиентный спуск на плоскости")|

![Градиентный спуск](https://upload.wikimedia.org/wikipedia/commons/6/68/Gradient_ascent_%28surface%29.png "Градиентный спуск в пространстве")

![Градиентный спуск](https://upload.wikimedia.org/wikipedia/commons/f/f3/Stogra.png "Поведение функции потерь градиентного спуска (ошибки модели)")

Алгоритм заключается в итеративном подстраивании весов модели с использованием входного и выходного наборов данных: на каждом шаге алгоритм вычисляет предсказание модели на одном примере из обучающей выборки, после чего вычисляет *ошибку*, используя выходной набор данных (реальные значения). Из этой ошибки формируется **антиградиент**

Формула (матричный вид):

$\huge W_t=W_{t-1}+\eta \nabla Q(W)$,

где:

* $\large -\nabla Q(W)=\{\frac{\partial Q}{\partial w_1}, \frac{\partial Q}{\partial w_2}, ..., \frac{\partial Q}{\partial w_n} \}$ - **антиградиент** ошибки относительно весов (параметров) W;
* $\large W_t$ - новая матрица (вектор) весов;
* $\large W_{t-1}$ - старая матрица (вектор) весов;
* $\large \eta$ - скорость обучения;
* $\large n$ - количество весов (параметров) в W.

Формула для $Q$ определяется выбранным функционалом ошибки (критерием потерь). Например, для MSE критерий ошибки будет:

$\huge Q=\frac{1}{n}\sum_{i=1}^{n} (y_{i}-g_{i})^2$,

где:

$\large n$ - количество значений (семплов);

$\large y_{i}$ - предсказание из i-го индекса в наборе всех значений;   

$\large g_{i}$ - реальное значение из i-го индекса в наборе всех значений.

Отсюда, принимая во внимание, что $y_{i}=f(x_{i})$ - формула модели, каждая из компонет $\nabla Q(W)$, будет:

$\huge \frac{\partial Q}{\partial w_i}=(f'(x_{i}))(f(x_{i})-g_{i})$

или в матричном виде:

$\huge \nabla Q=(f'(x))(f(x)-g)=\frac{2}{n}(f'(x))\times E(x)$

Из зтого следует, что основной проблемой будет нахождение производной функции модели $f(x)$. Если она заранее известна, то все сводится к ее вычислению по готовой формуле. Если вы заранее *не знаете*, как она выглядит, то вам будет необходимо ([Решение этой проблемы подробно освещено в дискуссии на stackoverflow](https://stackoverflow.com/questions/9876290/how-do-i-compute-derivative-using-numpy)):

1. Использовать конечное дифференцирование:
    Допустим, функция `fun` - это наша исходная функция модели; Тогда в следующем коде:
    ```python
       def fun(x):
           return x**2 + 1
       
       def d_fun(x):
           h = 1e-5
           return (fun(x+h)-fun(x-h))/(2*h)
    ```
    функция `d_fun` - это конечная производная (аппроксимация) с небольшим приращением `h`.
    
2. Использовать "хитрую" функцию дифференцирования из стороннего пакета (numpy по умолчанию такой функции не содержит):
    Например, пакет "autograd" позволяет дифференцировать произвольные функции. Установите его командой в консоли:
    ```bash
    pip install autograd
    ```
    После, используйте ее следующим образом (обратите внимание на `grad`):
    ```python
       import numpy as np
       from autograd import grad
        
       def fct(x):
           y = x**2+1
           return y
        
       grad_fct = grad(fct)
       print(grad_fct(1.0))
    ```
    
Достоинства:
- Много параметров для настройки (можно оптимизировать модель);
- Хорош для большого кол-ва данных (>10,000).

Недостатки:
- чувситвителен к размерности входных данных: необходимо *нормировать* данные.
- множество неявных побочных эффектов (переобучение, паралич, и т.д.)

[Википедия](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

[Доументация (Обучающийся регрессор)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor)

In [1]:
# Создание простейшей обучаемой модели в sklearn
import numpy as np
from sklearn import linear_model
n_samples, n_features = 10, 5
np.random.seed(0)
y = np.random.randn(n_samples)
X = np.random.randn(n_samples, n_features)
clf = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
print(clf.fit(X, y))

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=1000, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=0.001, verbose=0,
       warm_start=False)


In [8]:
# предсказание
clf.predict(X)

array([ 0.59113601,  0.52024237,  1.31387889,  1.39609664,  0.52466969,
        0.1215026 ,  1.05637942, -0.13433001,  0.73800374,  0.15920071])

In [11]:
# изменение размерности данных - необходимо перед обучением модели
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)  # Don't cheat - fit only on training data
X_train = scaler.transform(X)

In [3]:
# конечное дифференцирование
import numpy as np
from autograd import grad

def fct(x):
    y = x**2+1
    return y

grad_fct = grad(fct)
print(grad_fct)
print(grad_fct(1.0))

<function unary_to_nary.<locals>.nary_operator.<locals>.nary_f at 0x7f37a5d3be18>
2.0
